Задание 1

In [ ]:
from collections import namedtuple, defaultdict, deque, Counter, OrderedDict
from typing import List
import json, pickle

Book = namedtuple("Book", ["title", "author", "isbn", "year"])
Reader = namedtuple("Reader", ["name", "reader_id", "phone"])

list_of_books: List[str]= [
    Book(title="abc", author="111", isbn=1, year=2025),
    Book(title="bcd", author="222", isbn=2, year=2021),
    Book(title="cde", author="111", isbn=3, year=1234),
    Book(title="abc", author="333", isbn=4, year=5555),
]
books_by_genre: defaultdict = defaultdict(list)
books_by_genre["genre1"].append(list_of_books[0])
books_by_genre["genre2"].append(list_of_books[1])
books_by_genre["genre3"].append(list_of_books[2])
books_by_genre["genre2"].append(list_of_books[3])
deque_of_readers: deque = deque()
deque_of_readers.append(Reader(name="r1", reader_id=1, phone=789))
deque_of_readers.append(Reader(name="r2", reader_id=11, phone=000))
deque_of_readers.append(Reader(name="r5", reader_id=5, phone=23))
my_counter: Counter = Counter([i.author for i in list_of_books])
my_ordered_dict: OrderedDict = OrderedDict.fromkeys(deque_of_readers)
my_ordered_dict[Reader(name="r1", reader_id=1, phone=789)] = list_of_books[0]
my_ordered_dict[Reader(name="r5", reader_id=5, phone=23)] = list_of_books[3]


with open("task_1.json", "w") as file:
    json.dump(list_of_books, file)
    json.dump(deque_of_readers, file)
    json.dump(my_counter, file)
    json.dump(my_ordered_dict, file)
with open("task_1.pickle", "wb") as file:
    pickle.dump("list of books: " + str(list_of_books), file)
    pickle.dump("deque of reader: " + str(deque_of_readers), file)
    pickle.dump("counter of authors: " + str(my_counter), file)
    pickle.dump("ordered dict: " + str(my_ordered_dict), file)

Задание 2

In [ ]:
from collections import namedtuple, defaultdict, deque, Counter
import json, os, sys
from datetime import datetime

FileInfo: namedtuple = namedtuple("FileInfo", ["name", "size", "extension", "modified_time"])

files_by_size: defaultdict = defaultdict(list)
files_list: List = []
last_ten_files: deque = deque(maxlen=10)

counter_files_extentions: Counter = Counter()

for root, dirs, files in os.walk("."):
    for file in files:
        file_path = os.path.join(root, file)

        try:
            size = sys.getsizeof(file_path)
            mtime = os.path.getmtime(file_path)
            modified_time = datetime.fromtimestamp(mtime).strftime("%Y-%m-%d %H:%M:%S")
            name, extension = os.path.splitext(file)
            if not extension:
                extension = "no_extension"

            file_info: namedtuple = FileInfo(
                name=file, size=size, extension=extension, modified_time=modified_time
            )
            files_list.append(file_info)

            counter_files_extentions[extension] += 1

            if size < 1024 * 1024:
                files_by_size["small"].append(file_info)
            elif size <= 100 * 1024 * 1024:
                files_by_size["medium"].append(file_info)
            else:
                files_by_size["large"].append(file_info)

            last_ten_files.append(file_info)

        except (OSError, PermissionError):
            continue
with open("task_2.json", "w") as file:
    json.dump("list of files: " + str(files_list), file)
    json.dump("count ext: " + str(counter_files_extentions), file)
    json.dump("size groups: " + str(files_by_size), file)
    json.dump("last_10_files: " + str(last_ten_files), file)


Задание 3

In [ ]:
from collections import namedtuple, defaultdict, ChainMap, OrderedDict
import os, json, pickle

Config: namedtuple = namedtuple("Config", ["key", "value", "section", "default_value"])
default: defaultdict = defaultdict(list)
default["database.host"].append("localhost")
default["database.port"].append(5432)
default["app.debug"].append(False)
default["app.timeout"].append(30)
default["logging.level"].append("INFO")

user: defaultdict = defaultdict(list)
user["database.host"].append("127.0.0.1")
user["app.debug"].append(True)
user["user.name"].append("admin")
environment: defaultdict = defaultdict()
for key, value in os.environ.items():
    if key.startswith("APP_"):
        environment[key[4:].lower()] = value
prioritized_chain = ChainMap(default, user, environment)


grouped_config: defaultdict = defaultdict(dict)

for full_key in prioritized_chain:
    value = prioritized_chain[full_key]

    if "." in full_key:
        section, key = full_key.split(".", 1)
    else:
        section = "general"
        key = full_key

    grouped_config[section][key] = value

ordered_config: OrderedDict = OrderedDict()
ordered_config["environment"] = dict(environment)
ordered_config["user"] = dict(user)
ordered_config["default"] = dict(default)

with open("task_3.json", "w") as file:
    json.dump("default: " + str(default), file)
    json.dump("user: " + str(user), file)
    json.dump("environment: " + str(environment), file)
    json.dump("proritized_chain: " + str(prioritized_chain), file)
    json.dump("grouped_config: " + str(grouped_config), file)
    json.dump("ordered_config: " + str(ordered_config), file)
with open("task_3.pickle", "wb") as file:
    pickle.dump("default: " + str(default), file)
    pickle.dump("user: " + str(user), file)
    pickle.dump("environment: " + str(environment), file)
    pickle.dump("proritized_chain: " + str(prioritized_chain), file)
    pickle.dump("grouped_config: " + str(grouped_config), file)
    pickle.dump("ordered_config: " + str(ordered_config), file)


Задание 4

In [ ]:
from collections import namedtuple, deque, Counter, defaultdict
import datetime
from typing import NamedTuple
import os, sys, pickle

SystemInfo = namedtuple(
    "SystemInfo", ["cpu_count", "memory_usage", "process_id", "user_name"]
)

history: deque = deque(maxlen=20)
function_counter = Counter()
grouped_by_time: defaultdict = defaultdict(list)


def collect_system_info():
    function_counter["os.cpu_count"] += 1
    cpu_count = os.cpu_count()

    function_counter["sys.getallocatedblocks"] += 1
    memory_usage = sys.getallocatedblocks()

    function_counter["os.getpid"] += 1
    process_id = os.getpid()

    function_counter["os.getlogin"] += 1
    user_name = os.getlogin()

    return SystemInfo(cpu_count, memory_usage, process_id, user_name)


for _ in range(25):
    info = collect_system_info()
    current_time = datetime.now()

    history.append((current_time, info))

    hour_key = current_time.replace(minute=0, second=0, microsecond=0)
    grouped_by_time[hour_key].append(info)

with open("task_4.pickle", "wb") as file:
    pickle.dump(
        {
            "history": history,
            "function_counter": function_counter,
            "grouped_by_time": dict(grouped_by_time),
        },
        file,
    )

Задание 5

In [ ]:
from collections import namedtuple, deque, Counter, defaultdict, OrderedDict, ChainMap
from typing import NamedTuple, ChainMap
from datetime import datetime
import os, json, pickle


LogEntry: namedtuple = namedtuple(
    "LogEntry", ["timestamp", "level", "message", "module", "function"]
)
deque_for_logs = deque(maxlen=100)
dict_level = defaultdict(list)
count_log_level = Counter(dict_level)
logs_by_time = OrderedDict()
all_logs = ChainMap(
    {"recent": list(deque_for_logs)},
    dict(dict_level),
    {"counter": dict(count_log_level)},
    dict(logs_by_time),
)


def add_log(level, message, module, function):
    timestamp = datetime.now()
    log_entry = LogEntry(timestamp, level, message, module, function)

    deque_for_logs.append(log_entry)
    dict_level[level].append(log_entry)
    count_log_level[level] += 1
    logs_by_time[timestamp] = log_entry

    return log_entry


add_log("INFO", "111", "222", "333")
add_log("DEBUG", "22", "33", "55")
add_log("ERROR", "777", "2", "4")
add_log("WARNING", "5", "4", "2")

log_dir = "logs"
os.makedirs(log_dir, exist_ok=True)

json_file = os.path.join(log_dir, "task_5.json")
with open(json_file, "w") as f:
    json.dump([entry._asdict() for entry in deque_for_logs], f, default=str)

pickle_file = os.path.join(log_dir, "task_5.pickle")
with open(pickle_file, "wb") as f:
    pickle.dump(
        {
            "deque": deque_for_logs,
            "by_level": dict(dict_level),
            "counter": count_log_level,
            "by_time": logs_by_time,
        },
        f,
    )

Задание 6

In [2]:
from collections import namedtuple, deque, Counter, defaultdict, OrderedDict, ChainMap
from typing import NamedTuple, ChainMap
from datetime import datetime
import os, json, pickle, sys

CacheEntry: namedtuple = namedtuple(
    "CacheEntry", ["key", "value", "timestamp", "access_count"]
)
LRU_cache: OrderedDict = OrderedDict()
history_keys: deque = deque()
count_keys: Counter = Counter()
access_default = defaultdict(int)
MAX_CACHE_SIZE = 5


def add_to_cache(key, value):
    if len(LRU_cache) >= MAX_CACHE_SIZE:
        oldest_key, oldest_value = LRU_cache.popitem(last=False)
    new_entry = CacheEntry(
        key=key, value=value, timestamp=datetime.now(), access_count=0
    )

    LRU_cache[key] = new_entry


def get_from_cache(key):
    if key in LRU_cache:
        entry = LRU_cache[key]
        count_keys[key] += 1
        access_default[key] += 1
        history_keys.append((key, datetime.now()))
        new_entry = CacheEntry(
            key=key,
            value=entry.value,
            timestamp=entry.timestamp,
            access_count=entry.access_count + 1,
        )
        LRU_cache.move_to_end(key)
        LRU_cache[key] = new_entry

        return new_entry.value
    else:
        return None


def remove_from_cache(key):
    if key in LRU_cache:
        del LRU_cache[key]


def clear_cache():
    LRU_cache.clear()


def show_cache_size():
    size = len(LRU_cache)
    memory = sys.getsizeof(LRU_cache)


def save_cache(filename):
    with open(filename, "wb") as f:
        pickle.dump(dict(LRU_cache), f)


def load_cache(filename):
    with open(filename, "rb") as f:
        loaded_data = pickle.load(f)
        LRU_cache.update(loaded_data)


add_to_cache("user1", "a")
add_to_cache("user2", "b")
add_to_cache("user3", "c")
add_to_cache("user4", "d")
add_to_cache("user5", "e")

show_cache_size()

add_to_cache("user6", "f")

get_from_cache("user2")
get_from_cache("user1")

save_cache("task_6.pkl")
clear_cache()
show_cache_size()
load_cache("task_6.pkl")
show_cache_size()

remove_from_cache("user3")
show_cache_size()

Задание 7

In [ ]:
from collections import namedtuple, Counter, defaultdict, deque, OrderedDict
import os
import sys
import json
from datetime import datetime

WordInfo = namedtuple("WordInfo", ["word", "frequency", "length", "first_occurrence"])

text: str = "aa aaa ff f ff b c"
words: List[str] = text.split()

word_freq: Counter = Counter(words)
word_lengths: defaultdict = defaultdict(list)
for i in words:
    word_lengths[len(i)].append(i)

unique_words: deque = deque(maxlen=50)
for i in words:
    if i not in unique_words:
        unique_words.append(i)

word_order: OrderedDict = OrderedDict()
for i in words:
    if i not in word_order:
        word_order[i] = datetime.now()

word_stats = []
for word, freq in word_freq.items():
    word_stats.append(
        WordInfo(
            word=word,
            frequency=freq,
            length=len(word),
            first_occurrence=word_order[word],
        )
    )

memory_usage = sys.getsizeof(word_stats)
with open("task_7.json", "w") as f:
    json.dump([w._asdict() for w in word_stats], f, default=str)

Задание 8

In [ ]:
from collections import namedtuple, defaultdict, deque, Counter, OrderedDict, ChainMap
from typing import NamedTuple
import os
import json
import pickle
from datetime import datetime

Task: namedtuple = namedtuple(
    "Task", ["id", "title", "description", "priority", "status", "created_date"]
)
tasks_by_status: defaultdict = defaultdict(list)
priority_queue: deque = deque()
task_counter: Counter = Counter()
tasks_ordered: OrderedDict = OrderedDict()
all_tasks: ChainMap = ChainMap(
    tasks_by_status,
    {"priority_queue": list(priority_queue)},
    dict(task_counter),
    tasks_ordered,
)


def add_task(title, description, priority):
    task_id = len(tasks_ordered) + 1
    new_task = Task(
        id=task_id,
        title=title,
        description=description,
        priority=priority,
        status="todo",
        created_date=datetime.now(),
    )
    tasks_by_status["todo"].append(new_task)
    if priority == "high":
        priority_queue.append(new_task)
    task_counter[priority] += 1
    tasks_ordered[task_id] = new_task


def complete_task(task_id):
    if task_id in tasks_ordered:
        task = tasks_ordered[task_id]
        tasks_by_status[task.status].remove(task)
        tasks_by_status["done"].append(task._replace(status="done"))
        if task in priority_queue:
            priority_queue.remove(task)


def get_tasks_by_status(status):
    return tasks_by_status.get(status, [])


def get_priority_queue():
    return list(priority_queue)


with open("task_8.json", "w") as f:
    json.dump([t._asdict() for t in tasks_ordered.values()], f, default=str)
with open("task_8.pkl", "wb") as f:
    pickle.dump(dict(tasks_ordered), f)

Задание 9

In [ ]:
from collections import namedtuple, deque, defaultdict, Counter, OrderedDict
import sys
import os
import json
import pickle
from datetime import datetime

PerformanceMetric: namedtuple = namedtuple(
    "PerformanceMetric",
    ["function_name", "execution_time", "memory_usage", "timestamp"],
)
metrics_deque: deque = deque(maxlen=100)
metrics_by_function: defaultdict = defaultdict(list)
function_calls: Counter = Counter()
metrics_ordered: OrderedDict = OrderedDict()


def record_metric(function_name, execution_time, memory_usage):
    metric = PerformanceMetric(
        function_name=function_name,
        execution_time=execution_time,
        memory_usage=memory_usage,
        timestamp=datetime.now(),
    )
    metrics_deque.append(metric)
    metrics_by_function[function_name].append(metric)
    function_calls[function_name] += 1
    metrics_ordered[datetime.now()] = metric


def get_function_stats(function_name):
    return metrics_by_function.get(function_name, [])


def get_memory_usage():
    return (
        sys.getsizeof(metrics_deque)
        + sys.getsizeof(metrics_by_function)
        + sys.getsizeof(function_calls)
        + sys.getsizeof(metrics_ordered)
    )


def export_metrics():
    with open("task_9.json", "w") as f:
        json.dump([m._asdict() for m in metrics_deque], f, default=str)
    with open("task_9.pkl", "wb") as f:
        pickle.dump(list(metrics_deque), f)